## Notebook for Pre-processing (Step 1)
Reprojection (precip only) <br>
Resample to 1km resolution <br>
And Crop to AOIs

In [2]:
import sys
sys.path.append("../")

In [3]:
from osgeo import gdal
from model.GeoImgProc import GeoImgProc
import os
import numpy as np
import glob

In [21]:
import warnings
warnings.simplefilter("ignore")

In [4]:
## get srs from sinusoidal prj results ; this step can be skip when user provide srs upfront
def get_sin_srs():
    
    file = '/att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/CHIRPS/20150701sin.tif'
    ds = gdal.Open(file)
    prj = ds.GetProjection()
    
    return str(prj)

In [5]:
def compare_tifs(tif1, tif2):
    ds = gdal.Open(tif1)
    arr1 = ds.GetRasterBand(1).ReadAsArray()
    ds = None
    
    ds = gdal.Open(tif2)
    arr2 = ds.GetRasterBand(1).ReadAsArray()
    ds = None
    
    return np.array_equal(arr1, arr2, equal_nan=True)

# Change path to point to your geoTiffs, shp files in next cell 

In [6]:

data_base = "/att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out"

t_srs = get_sin_srs() ## modify if user provide prj

# cutline shp file for AOI 
aoi_shp = "/att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/AOI_sin_clipto.shp"

# cutline shp files for Senegal
regions = ['delta', 'podor', 'west']
pre = "/att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/senegal_"
post = ".shp"
senegal_shp = [pre+sub+post for sub in regions]


In [23]:
# process Precip data (based on reprojCHIRPS.sh & clipto*.sh)

## directory containing CHIRPS tifs
dirn = f"{data_base}/PRECIP"
f_list = glob.glob(os.path.join(dirn, '*2006????*.tif'))

## loop through CHIRPS file
for f in f_list:
    
    # 1 & 2. re-project to sinusoidal & re-sample to 1km resolution
    ofn = os.path.basename(f)
    ofn = os.path.splitext(ofn)[0]+".sin.tif"
    outfile1 = os.path.join(dirn, ofn)

    #warp_opts = {'-s_srs' : 'WGS84',
    #             '-t_srs' : t_srs,
    #             '-r' : 'bilinear',
    #            '-tr' : '926.625433005 926.625433005',
    #             '-tap' : ''}
    warp_opts = {'-s_srs' : 'WGS84',
                '-t_srs' : t_srs,
                '-r' : 'mode'}
    GeoImgProc.warp(f, outfile1, **warp_opts)
    print("Re-projection Completed")
    
    # 2. re-sample to 1km resolution
    ofn = os.path.basename(f)
    ofn = os.path.splitext(ofn)[0]+".1km.tif"
    outfile2 = os.path.join(dirn, ofn)
    
    warp_opts = {'-tr' : '926.625433005 926.625433005',
                 '-r' : 'bilinear'}
    
    GeoImgProc.warp(outfile1, outfile2, **warp_opts)
    print("Re-sample Completed")
    
    # 3. crop to AOI
    ofn = os.path.basename(f)
    ofn = os.path.splitext(ofn)[0]+".clip.tif"
    outfile3 = os.path.join(dirn, ofn)
    
    warp_opts ={'-crop_to_cutline' : '',
                '-cutline' : aoi_shp}
    
    GeoImgProc.warp(outfile2, outfile3, **warp_opts)
    print("Crop Completed")
    
    # 4. crop to Senegal
    for i,aoi in enumerate(regions):
        
        ofn = os.path.basename(outfile3)
        ofn = os.path.splitext(ofn)[0]+f"{aoi}.tif"
        outfile4 = os.path.join(dirn, ofn)
        
        s_shp = senegal_shp[i]
        
        warp_opts ={'-crop_to_cutline' : '',
                '-cutline' : s_shp}
        
        GeoImgProc.warp(outfile3, outfile4, **warp_opts)

ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PRO

Creating output file that is 2846P x 2846L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060310-S000000-E235959.V06.precip.tif [1/1] : 0Using internal nodata values (e.g. -9999.9) for image /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060310-S000000-E235959.V06.precip.tif.
Copying nodata values from source /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060310-S000000-E235959.V06.precip.tif to destination /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060310-S000000-E235959.V06.precip.sin.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Re-projection Completed
Creating output file that is 21600P x 21600L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060310-S000000-E235959.V06.precip.sin.t

ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PRO

Creating output file that is 2846P x 2846L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060319-S000000-E235959.V06.precip.tif [1/1] : 0Using internal nodata values (e.g. -9999.9) for image /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060319-S000000-E235959.V06.precip.tif.
Copying nodata values from source /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060319-S000000-E235959.V06.precip.tif to destination /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060319-S000000-E235959.V06.precip.sin.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Re-projection Completed
Creating output file that is 21600P x 21600L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060319-S000000-E235959.V06.precip.sin.t

ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PRO

...

Creating output file that is 2846P x 2846L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060317-S000000-E235959.V06.precip.tif [1/1] : 0Using internal nodata values (e.g. -9999.9) for image /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060317-S000000-E235959.V06.precip.tif.
Copying nodata values from source /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060317-S000000-E235959.V06.precip.tif to destination /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060317-S000000-E235959.V06.precip.sin.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Re-projection Completed
Creating output file that is 21600P x 21600L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060317-S000000-E235959.V06.precip.sin.t

ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PRO

Creating output file that is 2846P x 2846L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060307-S000000-E235959.V06.precip.tif [1/1] : 0Using internal nodata values (e.g. -9999.9) for image /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060307-S000000-E235959.V06.precip.tif.
Copying nodata values from source /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060307-S000000-E235959.V06.precip.tif to destination /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060307-S000000-E235959.V06.precip.sin.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Re-projection Completed
Creating output file that is 21600P x 21600L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060307-S000000-E235959.V06.precip.sin.t

ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PROJ: sinu: Invalid latitude
ERROR 1: PRO

Creating output file that is 2846P x 2846L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060314-S000000-E235959.V06.precip.tif [1/1] : 0Using internal nodata values (e.g. -9999.9) for image /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060314-S000000-E235959.V06.precip.tif.
Copying nodata values from source /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060314-S000000-E235959.V06.precip.tif to destination /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060314-S000000-E235959.V06.precip.sin.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Re-projection Completed
Creating output file that is 21600P x 21600L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/PRECIP/3B-DAY-L.MS.MRG.3IMERG.20060314-S000000-E235959.V06.precip.sin.t

In [16]:
# process NDVI data (based on resizeNDVI.sh & clipto*.sh)

## directory containing NDVI tifs
dirn = f"{data_base}/NDVI"
f_list = glob.glob(os.path.join(dirn, '*.tif'))
#print(f_list)

## loop through NDVI files
for f in f_list:
        
    # 1. re-sample to 1km resolution
    ofn = os.path.basename(f)
    ofn = os.path.splitext(ofn)[0]+".1km.tif"
    outfile1 = os.path.join(dirn, ofn)
    
    warp_opts = {'-tr' : '926.625433005 926.625433005',
                 '-r' : 'mode'}
    
    GeoImgProc.warp(f, outfile1, **warp_opts)
    print("Re-sample Completed")
    
    # 2. crop to AOI
    ofn = os.path.basename(f)
    ofn = os.path.splitext(ofn)[0]+".clip.tif"
    outfile2 = os.path.join(dirn, ofn)
    
    warp_opts ={'-crop_to_cutline' : '',
                '-cutline' : aoi_shp}
    
    GeoImgProc.warp(outfile1, outfile2, **warp_opts)
    print("Crop Completed")
    
    # 3. crop to Senegal
    for i,aoi in enumerate(regions):
        
        ofn = os.path.basename(outfile2)
        ofn = os.path.splitext(ofn)[0]+aoi+".tif"
        outfile3 = os.path.join(dirn, ofn)
        
        s_shp = senegal_shp[i]
        
        warp_opts ={'-crop_to_cutline' : '',
                '-cutline' : s_shp}
        
        GeoImgProc.warp(outfile2, outfile3, **warp_opts)

Creating output file that is 1200P x 1200L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/NDVI/MOD13Q1.A2003081.h16v07.006.2015156190635.ndvi.tif [1/1] : 0Using internal nodata values (e.g. -3000) for image /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/NDVI/MOD13Q1.A2003081.h16v07.006.2015156190635.ndvi.tif.
Copying nodata values from source /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/NDVI/MOD13Q1.A2003081.h16v07.006.2015156190635.ndvi.tif to destination /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/NDVI/MOD13Q1.A2003081.h16v07.006.2015156190635.ndvi.1km.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Re-sample Completed
Creating output file that is 336P x 125L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/NDVI/MOD13Q1.A2003081.h16v07.006.2015156190635.ndvi.1km.tif [1/1] : 0Using internal nodata values (e.g. -3000) for image /att/nobackup/jl

In [15]:
# process Landcover data (based on resizeLANDCOVER.sh & clipto*.sh)

## directory containing NDVI tifs
dirn = f"{data_base}/LANDCOVER"
f_list = glob.glob(os.path.join(dirn, '*.tif'))
#print(f_list)

## loop through LC files
for f in f_list:
        
    # 1. re-sample to 1km resolution
    ofn = os.path.basename(f)
    ofn = os.path.splitext(ofn)[0]+".1km.tif"
    outfile1 = os.path.join(dirn, ofn)
    
    warp_opts = {'-tr' : '926.625433005 926.625433005',
                 '-r' : 'mode'}
    
    GeoImgProc.warp(f, outfile1, **warp_opts)
    print("Re-sample Completed")
    
    # 2. crop to AOI
    ofn = os.path.basename(f)
    ofn = os.path.splitext(ofn)[0]+".clip.tif"
    outfile2 = os.path.join(dirn, ofn)
    
    warp_opts ={'-crop_to_cutline' : '',
                '-cutline' : aoi_shp}
    
    GeoImgProc.warp(outfile1, outfile2, **warp_opts)
    print("Crop Completed")
    
    # 3. crop to Senegal
    for i,aoi in enumerate(regions):
        
        ofn = os.path.basename(outfile2)
        ofn = os.path.splitext(ofn)[0]+aoi+".tif"
        outfile3 = os.path.join(dirn, ofn)
        
        s_shp = senegal_shp[i]
        
        warp_opts ={'-crop_to_cutline' : '',
                '-cutline' : s_shp}
        
        GeoImgProc.warp(outfile2, outfile3, **warp_opts)

Creating output file that is 1200P x 1200L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/LANDCOVER/MCD12Q1.A2006001.h16v07.006.2018145201602.landcover.tif [1/1] : 0Using internal nodata values (e.g. 255) for image /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/LANDCOVER/MCD12Q1.A2006001.h16v07.006.2018145201602.landcover.tif.
Copying nodata values from source /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/LANDCOVER/MCD12Q1.A2006001.h16v07.006.2018145201602.landcover.tif to destination /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/LANDCOVER/MCD12Q1.A2006001.h16v07.006.2018145201602.landcover.1km.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Re-sample Completed
Creating output file that is 336P x 125L.
Processing /att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/LANDCOVER/MCD12Q1.A2006001.h16v07.006.2018145201602.landcover.1km.tif [1/1] : 0Using internal nodat